<a href="https://colab.research.google.com/github/Himanshu0702/Machine-Learning/blob/main/Tlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import cv2
import PIL
import PIL.Image as Image
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [75]:
data_dir = '/content/drive/MyDrive/Skin_Disease'

import pathlib 
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/content/drive/MyDrive/Skin_Disease')

In [76]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3592


In [77]:
diseases_images_dict = {
    'Melanoma' : list(data_dir.glob('Melanoma Skin Cancer Nevi and Moles/*')),
    'Eczema' : list(data_dir.glob('Eczema Photos/*')),
    'Dermatitis' : list(data_dir.glob('Atopic Dermatitis Photos/*')),
    'Psoriasis' : list(data_dir.glob('Psoriasis pictures Lichen Planus and related diseases/*')),
}

In [78]:
diseases_labels_dict = {
    'Melanoma' : 0,
    'Eczema' : 1,
    'Dermatitis': 2,
    'Psoriasis': 3,
}

In [79]:
train = '/content/drive/MyDrive/dataset/train'
test = '/content/drive/MyDrive/dataset/test'
validation = '/content/drive/MyDrive/dataset/val'

In [93]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

    #splits data into training and testing(validation) sets
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/dataset/train',  # Source directory
        target_size=(224, 224),  # Resizes images
        batch_size=15,
        class_mode='sparse')

test_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/dataset/test',  # Source directory
        target_size=(224, 224),  # Resizes images
        batch_size=15,
        class_mode='sparse')
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/val',
    target_size=(224, 224),
    batch_size=15,
    class_mode='sparse')

Found 2513 images belonging to 4 classes.
Found 362 images belonging to 4 classes.
Found 717 images belonging to 4 classes.


In [81]:
feature_extractor = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [82]:
feature_extractor_layer = hub.KerasLayer(feature_extractor, input_shape=(224,224,3))

In [83]:
feature_extractor_layer.trainable = False

In [84]:
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [85]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])


In [87]:
history = model.fit(train_generator, epochs=6, validation_data = validation_generator)

Epoch 1/6
168/168 [==============================] - 116s 692ms/step - loss: 0.7478 - acc: 0.6848 - val_loss: 0.8232 - val_acc: 0.6695
Epoch 2/6
168/168 [==============================] - 117s 696ms/step - loss: 0.7192 - acc: 0.7023 - val_loss: 0.8115 - val_acc: 0.6722
Epoch 3/6
168/168 [==============================] - 123s 733ms/step - loss: 0.7210 - acc: 0.6892 - val_loss: 0.8281 - val_acc: 0.6625
Epoch 4/6
168/168 [==============================] - 118s 700ms/step - loss: 0.6913 - acc: 0.7127 - val_loss: 0.8277 - val_acc: 0.6778
Epoch 5/6
168/168 [==============================] - 116s 693ms/step - loss: 0.7003 - acc: 0.7119 - val_loss: 0.8503 - val_acc: 0.6388
Epoch 6/6
168/168 [==============================] - 118s 699ms/step - loss: 0.6763 - acc: 0.7087 - val_loss: 0.7926 - val_acc: 0.6750


In [96]:
result = model.evaluate(test_generator)

25/25 [==============================] - 13s 522ms/step - loss: 0.8121 - acc: 0.6713


In [99]:
img = cv2.imread('/content/drive/MyDrive/dataset/test/Melanoma Skin Cancer Nevi and Moles/atypical-nevi-15.jpg')
resize = tf.image.resize(img, (224,224))
yhat = model.predict(np.expand_dims(resize/255,0))
yhat

1/1 [==============================] - 0s 58ms/step


array([[0.01032621, 0.0089736 , 0.90641207, 0.07428809]], dtype=float32)

In [98]:
train_generator.class_indices

{'Atopic Dermatitis Photos': 0,
 'Eczema Photos': 1,
 'Melanoma Skin Cancer Nevi and Moles': 2,
 'Psoriasis pictures Lichen Planus and related diseases': 3}